# City Budget Extractor

In [24]:
import PyPDF2
import pandas as pd

After struggling for 20 minutes or so realized that the numbering in the PDF document don't match what is pulled by PyPDf2 because the budget PDF includes some "intro" pages that aren't counted by the GUI pdf reader but are counted by PyPDF2

In [6]:
# This PDF has some special formatting that offsets the page numbers
page_offset = 11
filename = "FY-19-20-Adopted-Budget.pdf"

reader = PyPDF2.PdfFileReader(filename)
page = reader.getPage(279 +11)

Lets take a look at what the extracted text looks like. Because its a PDF Im already expecting something terrible and that what is looks like we got. We back one large string that sort of goes across the page row by row. We'll need to split this apart using some custom logic,

In [7]:
text = page.extractText()

pprint.pprint(text[:400])

('Expenditure Object Account\n'
 'FY 2015/16\n'
 'Actual\n'
 'FY 2016/17\n'
 'Actual\n'
 'FY 2017/18 \n'
 'Actual\n'
 'FY 2018/19 \n'
 'Adopted\n'
 'FY 2018/19 \n'
 'Revised\n'
 'FY 2019/20 \n'
 'Adopted\n'
 'Percent \n'
 'Change From \n'
 'Prior Year\n'
 ' All Funds\n'
 '   PERSONNEL SERVICES\n'
 '    Salaries, Permanent\n'
 '34,323,749\n'
 ' 34,654,406\n'
 ' 25,765,375\n'
 ' 37,010,295\n'
 ' 37,033,765\n'
 ' 36,135,762\n'
 ' -2.36%\n'
 '    Salaries, Temporary\n'
 '499,772\n'
 '      420,908\n'
 '      348,015\n'
 '      367,098\n'
 '      538,702\n'
 ' ')


For the headers I'm just going to write them down manually. While I could write some clever python it's really not worth it because there's only 6 headers, they're the same page to page, and I don't want the actual string from the text anyway.

With the headers done lets extract each row. The pattern I see here is some string that tells us the fund type, an

1. String thas in  "All Funds", "General Fund
2. And all caps line that signifies the start of the block of expenses
3. The line item of expenses
4. 6 rows that are the actual budget expenses for my city



In [ ]:
class Parser:
    """Parses the PDF file to grab the city expednitures as dates
    Performs the work in three passes
    
    1. Identifying the Block Headers and closing lines
    2. Identifying the row titles
    3. Parsing the row values and determining which ones can be parsed to valid row
    
    """
    
    HEADERS = ["FY2015/16_Actual", "FY2016/17_Actual",
                         "FY2017/18_Actual",  "FY2018/19_Actual",
                         "FY2018/19_Revised", "FY2019/20_Adopted"]
    
    def __init__(self, page=279, page_offset=11, filename="FY-19-20-Adopted-Budget.pdf"):
        """Gets page from pdf and page text values
        
        Notes
        -----
        This PDF has some special formatting that offsets the page numbers

        filename = "FY-19-20-Adopted-Budget.pdf"
        """
        self.page = reader.getPage(279 +11)
        
        # Split the text into discrete word units and clean up spacing
        
        # List Comprehension
        self.text = tuple([line.strip() for line in self.page.extractText().split("\n")])
        
        
        self.text = pd.Series(self.text)
        
        print("New Val")
        # Parse the budget numbers into numbers
        self.text = self.text.apply(self.coerce_numbers)
        
        
    def parse(self):
        raise NotImplementedError
        
    @staticmethod
    def is_header(line):
        
        if isinstance(line, int):
            return False
        else:
            return line.isupper() and "".join(line.split(" ")).isalpha()
    
    @staticmethod
    def coerce_numbers(line):
        """Tries parsing number strings into numbers, else return string"""
        
        # Check if number is negative with parantheses
        if line[0] == "(" and line[-1] ==")":
            neg_number = int("".join(line[1:-1].split(",")))
            breakpoint()
            return neg_number
        
        # Otherwise try plain logic
        try:
            return int("".join(line.split(",")))
        except ValueError:
            return line
        
    def parse_block_headers(self):
        """Identify the headers from the page as well as ending line"""
        current_header = {}
        headers = []
        
        for i, line in self.text.iteritems():
            if self.is_header(line):
                if line != current_header.get("header"):
                    current_header = {"header":line, "start":i}
                else:
                    assert line == current_header.get("header")
                    current_header["end"] = i
                    headers.append(current_header)
                    current_header = {}
            
        self.headers = pd.DataFrame(headers)
        return self.headers
        
    def parse_row_labels(self):
        """Identify the row labels from the page 
        
        Notes
        ----
        Row labels must appear after first header and before last header
        
        """
        
        rows = []
        for i, (header, start, end) in df.iterrows():
            
            row = {"header":header, "values":[]}
            
            # Get text block for this budget item block
            text_block = self.text[start+1:end]
            
            
            for line in text_block:
                
                # All rows should end with a % sign
                if "%" in str(line):
                    if len(row["values"]) == 7:
                        row["complete"] = True
                    else:
                        row["complete"] = False

                    # Explode out the values using the headers
                    numbers = row.pop("values")
                    for key, val in zip(self.HEADERS, numbers):
                        row[key] = val

                    rows.append(row)
                    row = {"header":header, "values":[]}
                elif isinstance(line, str):
                    row["line_item"] = line
                    
                else:
                    assert isinstance(line, int)
                    row["values"].append(line)
            
        self.budget = pd.DataFrame(rows)
        return self.budget

        
    def parse_numbers(self):
        """Identify indices of valid numbers from the page"""

In [ ]:
p = Parser()
df = p.parse_block_headers()
p.parse_row_labels()

In [85]:
p = Parser()
p.parse_block_headers()

New Val


,header,start,end
0,PERSONNEL SERVICES,17,54
1,OPERATING EXPENSES,62,143
2,CAPITAL EXPENDITURES,151,164


In [91]:
# Sentinel value for if I'm on a header row
header = None


def parse_budget(page_text, headers=HEADERS):
    """Given the page for a text parses out a table with budgetary information
    and returns a dataframe
    """
    rows = []
    text_lines = iter(text.split("\n"))
    header = None
  
    for line in text_lines:
        line = line.strip()
        
        # Deal with the header block and the closing line of a block
        # The header rows are uppercase but not alphanumeric so need both conditions to skip
        print(line)
        
        if line.isupper() and "".join(line.split(" ")).isalpha():
            # breakpoint()

            # This is closing of budget type block
            if line == header:
                print(f"In closing line of block {header}")
            
            else:
                print(f"In opening line of block {header}")
                header = line
                header_row = True
            continue
        
        # TODO: Remove this to stop the early break for testing
        elif "PERSONNEL SERVICES" in line:
        #if "total" in line.lower():
            break
        
        # Is not none because this will skip the heaers and crap at the top of the page
        elif header is not None:
            # breakpoint()

            row = []
            row.append(header)


            line_item = line
            row.append(line_item)
            
            print(f"Iterating over line items {line_item}")
            # Iterate over all the numerical items
            for _ in HEADERS:
                
                budget_value = next(text_lines)
                row.append(budget_value)
                #breakpoint()
            
            print(row)
            rows.append(row)
            
            # Skip the percentage column
            next(text_lines)
            
    df = pd.DataFrame(rows, columns=HEADERS)
    return df

In [90]:
text = page.extractText()

HEADERS = ["FY2015/16_Actual", "FY2016/17_Actual", 
           "FY2017/18_Actual",  "FY2018/19_Actual", 
           "FY2018/19_Revised", "FY2019/20_Adopted"]

parse_budget(text, HEADERS)

Expenditure Object Account
FY 2015/16
Actual
FY 2016/17
Actual
FY 2017/18
Actual
FY 2018/19
Adopted
FY 2018/19
Revised
FY 2019/20
Adopted
Percent
Change From
Prior Year
All Funds
PERSONNEL SERVICES
In opening line of block None
Salaries, Permanent
Iterating over line items Salaries, Permanent
['PERSONNEL SERVICES', 'Salaries, Permanent', '34,323,749', ' 34,654,406', ' 25,765,375', ' 37,010,295', ' 37,033,765', ' 36,135,762']
Salaries, Temporary
Iterating over line items Salaries, Temporary
['PERSONNEL SERVICES', 'Salaries, Temporary', '499,772', '      420,908', '      348,015', '      367,098', '      538,702', '      367,948']
Salaries, Overtime
Iterating over line items Salaries, Overtime
['PERSONNEL SERVICES', 'Salaries, Overtime', '5,007,346', '   5,043,233', '   4,093,771', '   3,953,950', '   4,372,335', '   4,049,950']
Leave Payouts
Iterating over line items Leave Payouts
['PERSONNEL SERVICES', 'Leave Payouts', '1,466,088', '   1,550,479', '   1,079,615', '       Benefits', '25

StopIteration: 

In [10]:
text.split("\n")

['Expenditure Object Account',
 'FY 2015/16',
 'Actual',
 'FY 2016/17',
 'Actual',
 'FY 2017/18 ',
 'Actual',
 'FY 2018/19 ',
 'Adopted',
 'FY 2018/19 ',
 'Revised',
 'FY 2019/20 ',
 'Adopted',
 'Percent ',
 'Change From ',
 'Prior Year',
 ' All Funds',
 '   PERSONNEL SERVICES',
 '    Salaries, Permanent',
 '34,323,749',
 ' 34,654,406',
 ' 25,765,375',
 ' 37,010,295',
 ' 37,033,765',
 ' 36,135,762',
 ' -2.36%',
 '    Salaries, Temporary',
 '499,772',
 '      420,908',
 '      348,015',
 '      367,098',
 '      538,702',
 '      367,948',
 '      0.23%',
 '    Salaries, Overtime',
 '5,007,346',
 '   5,043,233',
 '   4,093,771',
 '   3,953,950',
 '   4,372,335',
 '   4,049,950',
 '   2.43%',
 '    Leave Payouts ',
 '1,466,088',
 '   1,550,479',
 '   1,079,615',
 '       Benefits',
 '25,343,062',
 ' 26,926,178',
 ' 21,666,643',
 ' 29,692,140',
 ' 29,710,235',
 ' 32,459,909',
 ' 9.32%',
 '   PERSONNEL SERVICES',
 '66,640,018',
 ' 68,595,204',
 ' 52,953,418',
 ' 71,023,482',
 ' 71,655,037'